In [10]:
import torch
import torch.nn as nn
import os
from torch.utils.data import Dataset, DataLoader
from power_plant_dataset import PowerPlantDataset

# 定义模型
class PolynomialRegressor(nn.Module):
    def __init__(self, degree):
        super(PolynomialRegressor, self).__init__()
        self.degree = degree
        self.coefficients_mean = nn.Parameter(torch.randn(degree + 1))
        self.coefficients_std = nn.Parameter(torch.randn(degree + 1))

    def forward(self, x):
        mean = sum(self.coefficients_mean[i] * x.pow(i) for i in range(self.degree + 1))
        std = torch.exp(sum(self.coefficients_std[i] * x.pow(i) for i in range(self.degree + 1)))
        return mean, std

def predict_power_output(power_output, power_type):
    # 置信区间配置
    confidence_intervals = {
        'fd': {'lower': 2.0, 'upper': 2.0},
        'gf': {'lower': 1.5, 'upper': 2.5},
        'hdrm': {'lower': 1.2, 'upper': 1.2},
        'sdyg': {'lower': 0.5, 'upper': 1.0}
    }
    
    # 数据和模型路径配置
    dataset_file = f'fdcl/split/df_{power_type}.csv'
    model_file = f'fdcl/{power_type}.pth'
    
    # 加载数据集
    dataset = PowerPlantDataset(csv_file=dataset_file)
    
    # 初始化模型
    model = PolynomialRegressor(degree=3)
    
    # 加载模型参数
    if os.path.exists(model_file):
        model.load_state_dict(torch.load(model_file))
    else:
        print(f"模型参数文件 '{model_file}' 不存在.")
        return
    
    # 数据预处理
    normalized_input = dataset.scaler_features.transform([[power_output]])
    input_tensor = torch.from_numpy(normalized_input).float()
    
    # 进行预测
    with torch.no_grad():
        mean, std = model(input_tensor)
    
    # 逆归一化输出值
    output_mean = mean.numpy()
    output_std = std.numpy()
    predicted_mean = dataset.scaler_labels.inverse_transform(output_mean.reshape(1, -1))
    predicted_std = dataset.scaler_labels.inverse_transform(output_std.reshape(1, -1))
    
    # 计算置信区间
    ci = confidence_intervals[power_type]
    lower_bound = dataset.scaler_labels.inverse_transform((mean - ci['lower'] * std).numpy().reshape(1, -1))
    upper_bound = dataset.scaler_labels.inverse_transform((mean + ci['upper'] * std).numpy().reshape(1, -1))
    
    print(f"Predicted Mean: {predicted_mean}")
    print(f"Predicted Std Dev: {predicted_std}")
    print(f"Confidence Interval: {lower_bound} to {upper_bound}")

# 示例调用
predict_power_output(1350.33, 'hdrm')


Predicted Mean: [[15.281016]]
Predicted Std Dev: [[12.476927]]
Confidence Interval: [[10.264625]] to [[20.29741]]
